# Data Engineering

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import OrdinalEncoder

### Load Dataset

In [ ]:
flights = pd.read_csv("/work/Dataset_Post_Cleaning.csv", sep=",")

### Get total quantities

In [ ]:
desired_prod_types = ['Botanas', 'Licores', 'Galletas', 'Bebidas Calientes','Alimentos Charter', 'Perecederos', 'Refrescos', 'Sopas', 'Lacteos']
desired_prod_types = ["Quantity_"+i for i in desired_prod_types]

flights["Total_Quantity"] = flights[desired_prod_types].sum(axis=1)

### Change Time Strings to Datetime object

In [ ]:
flights['STD'] = flights['STD'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
flights['STA'] = flights['STA'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

### Flight Frequency by Days of the Week

In [ ]:
# Get days of the week
flights['DayoftheWeek'] = flights['STD'].dt.day_name()

### Flight Frequency by Weeks of the Year

In [ ]:
# Assuming 'flights' DataFrame already exists and 'STD' is the datetime column
flights['STD'] = pd.to_datetime(flights['STD'])

# Create a column with the week of the year
flights['WeekoftheYear'] = flights['STD'].dt.isocalendar().week
flights['Month'] = flights['STD'].dt.month

# Week-Month
flights['Week_Month_Label'] = flights.apply(
    lambda row: f"Week {row['WeekoftheYear']} ({datetime.strptime(str(row['Month']), '%m').strftime('%b')})",
    axis=1)

### Create Duration Intervals 

In [ ]:
longest_duration = np.ceil(flights["DUR_HR"].max()) + 1
flights["DUR_INTERVAL"] = pd.cut(x=flights["DUR_HR"], bins=np.arange(0,longest_duration,1), labels=[f"{i}" for i in np.arange(0,longest_duration-1,1)])

### Add Time interval of the day

We discretized the day into four interval.

In [ ]:
flights['Time_Interval'] = pd.cut(flights['STD'].dt.hour,
                                bins=[0, 6, 12, 18, 24],
                                labels=['00:00-06:00', '06:00-12:00', '12:00-18:00', '18:00-24:00'],
                                right=False)

In [ ]:
encoder_days = OrdinalEncoder()
encoder_time_intervals = OrdinalEncoder()
encoder_week_month = OrdinalEncoder()

flights[["DayoftheWeek_E"]] = encoder_days.fit_transform(flights[["DayoftheWeek"]])
flights[["Time_Interval_E"]] = encoder_time_intervals.fit_transform(flights[["Time_Interval"]])
flights[["Week_Month_Label_E"]] = encoder_week_month.fit_transform(flights[["Week_Month_Label"]])

days_map = {key:i for i, key in enumerate(encoder_days.categories_[0])}

time_map = {key:i for i, key in enumerate(encoder_time_intervals.categories_[0])}

week_month_map = {key:i for i, key in enumerate(encoder_week_month.categories_[0])}


print(days_map,time_map,week_month_map)

{'Friday': 0, 'Monday': 1, 'Saturday': 2, 'Sunday': 3, 'Thursday': 4, 'Tuesday': 5, 'Wednesday': 6} {'00:00-06:00': 0, '06:00-12:00': 1, '12:00-18:00': 2, '18:00-24:00': 3} {'Week 1 (Jan)': 0, 'Week 10 (Mar)': 1, 'Week 11 (Mar)': 2, 'Week 12 (Mar)': 3, 'Week 13 (Apr)': 4, 'Week 13 (Mar)': 5, 'Week 14 (Apr)': 6, 'Week 15 (Apr)': 7, 'Week 16 (Apr)': 8, 'Week 17 (Apr)': 9, 'Week 18 (May)': 10, 'Week 19 (May)': 11, 'Week 2 (Jan)': 12, 'Week 20 (May)': 13, 'Week 21 (May)': 14, 'Week 22 (Jun)': 15, 'Week 22 (May)': 16, 'Week 23 (Jun)': 17, 'Week 24 (Jun)': 18, 'Week 25 (Jun)': 19, 'Week 26 (Jul)': 20, 'Week 26 (Jun)': 21, 'Week 27 (Jul)': 22, 'Week 28 (Jul)': 23, 'Week 29 (Jul)': 24, 'Week 3 (Jan)': 25, 'Week 30 (Jul)': 26, 'Week 31 (Aug)': 27, 'Week 31 (Jul)': 28, 'Week 32 (Aug)': 29, 'Week 33 (Aug)': 30, 'Week 34 (Aug)': 31, 'Week 35 (Aug)': 32, 'Week 35 (Sep)': 33, 'Week 36 (Sep)': 34, 'Week 37 (Sep)': 35, 'Week 38 (Sep)': 36, 'Week 39 (Oct)': 37, 'Week 39 (Sep)': 38, 'Week 4 (Jan)': 39, 

### Resulting Data Frame

In [ ]:
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,...,Total_Quantity,DayoftheWeek,WeekoftheYear,Month,Week_Month_Label,DUR_INTERVAL,Time_Interval,DayoftheWeek_E,Time_Interval_E,Week_Month_Label_E
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,...,42.0,Thursday,42,10,Week 42 (Oct),2.0,06:00-12:00,4.0,1.0,42.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,186.0,...,15.0,Monday,27,7,Week 27 (Jul),3.0,00:00-06:00,1.0,0.0,22.0
2,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,...,0.0,Monday,26,6,Week 26 (Jun),0.0,12:00-18:00,1.0,2.0,21.0
3,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,...,22.0,Friday,6,2,Week 6 (Feb),1.0,06:00-12:00,0.0,1.0,57.0
4,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,...,2.0,Thursday,36,9,Week 36 (Sep),0.0,12:00-18:00,4.0,2.0,34.0


In [ ]:
flights.isna().sum()

Flight_ID                       0
Aeronave                        0
DepartureStation                0
ArrivalStation                  0
Destination_Type                0
Origin_Type                     0
STD                             0
STA                             0
Capacity                        0
Passengers                      0
Bookings                        0
STD-int                         0
STA-int                         0
DUR_HR                          0
DepartureStation_Encoded        0
ArrivalStation_Encoded          0
Destination_Encoded             0
Origin_Encoded                  0
Quantity_Alimentos Charter      0
Quantity_Bebidas Calientes      0
Quantity_Botanas                0
Quantity_Galletas               0
Quantity_Lacteos                0
Quantity_Licores                0
Quantity_Perecederos            0
Quantity_Refrescos              0
Quantity_Sopas                  0
TotalSales_Alimentos Charter    0
TotalSales_Bebidas Calientes    0
TotalSales_Bot

# Export data frame to CSV

In [ ]:
flights.to_csv("Dataset_Post_Engineer.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=25b1adb0-2b47-474f-a7b7-c06d723a95f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>